In [ ]:
sqoop import \  
  --connect jdbc:mysql://ms.itversity.com:3306/retail_db \  
  --username retail_user \  
  --password itversity \  
  --table order_items \  
  --warehouse-dir /user/haroldmei/sqoop_import/retail_db \  
  --num-mappers 1 \  
  --delete-target-dir  

In [ ]:
sqoop import \
  --connect jdbc:mysql://ms.itversity.com:3306/retail_db \
  --username retail_user \
  --password itversity \
  --table order_items \
  --warehouse-dir /user/haroldmei/sqoop_import/retail_db \
  --num-mappers 2 \
  --delete-target-dir \
  --as-textfile

In [ ]:
sqoop import \
  --connect jdbc:mysql://ms.itversity.com:3306/retail_db \
  --username retail_user \
  --password itversity \
  --table order_items \
  --warehouse-dir /user/haroldmei/sqoop_import/retail_db \
  --num-mappers 2 \
  --as-textfile \
  --compress \
  --delete-target-dir \
  --compression-codec org.apache.hadoop.io.compress.GzipCodec
    # org.apache.hadoop.io.compress.SnappyCodec

In [ ]:
hadoop fs -cat \
/user/haroldmei/sqoop_import/retail_db/order_items/part-m-00000.gz \
| gzip -d \
| hadoop fs -put - /user/haroldmei/sqoop_import/retail_db/order_items/demo.txt

#### Import all tables of a database to hdfs
sqoop import-all-tables \  
  --connect jdbc:mysql://ms.itversity.com:3306/retail_db \  
  --username retail_user \  
  --password itversity \  
  --warehouse-dir /user/haroldmei/sqoop_import/retail_db \  
  --autoreset-to-one-mapper 

#### Import all tables of a database to hive
sqoop import-all-tables \  
  --connect jdbc:mysql://ms.itversity.com:3306/retail_db \  
  --username retail_user \  
  --password itversity \  
  --hive-import \  
  --hive-database haroldmei_sqoop_import \  
  --autoreset-to-one-mapper   

#### Processing data with hive query
> create table daily_revenue as  
                             > select order_date,sum(order_item_subtotal) daily_revenue  
                             > from orders join order_items on  
                             > order_id=order_item_order_id  
                             > where order_date like '2013-07%'  
                             > group by order_date limit 10;  

#### Export hive table
--export-dir: the table location in hdfs

sqoop export --connect jdbc:mysql://ms.itversity.com:3306/retail_export \  
--username retail_user \  
--password itversity \  
--export-dir /apps/hive/warehouse/haroldmei_sqoop_import.db/daily_revenue \  
--input-fields-terminated-by "\001" \  
--table daily_revenue  


sqoop export --connect jdbc:mysql://ms.itversity.com:3306/retail_export \
--username retail_user --password itversity \
--export-dir /apps/hive/warehouse/haroldmei_sqoop_import.db/daily_revenue \
--table daily_revenue_demo \
--columns order_date,revenue \
--input-fields-terminated-by "\001" \
--num-mappers 1

In [ ]:
from pyspark import StorageLevel
lines = sc.textFile("/public/randomtextwriter/part-m-00000")
lines.persist(StorageLevel.MEMORY_ONLY)
lines.count()

In [ ]:
lines = sc.textFile("/public/randomtextwriter/part-m-00000")
def getWordTuples(i):
  import itertools as it
  # here plays the same role as spark flatMap
  wordTuples = map(lambda s: (s, 1), it.chain.from_iterable(map(lambda s: s.split(" "), i)))
  return wordTuples

wordTuples = lines.mapPartitions(lambda i: getWordTuples(i))
for i in wordTuples.reduceByKey(lambda x, y: x + y).take(10):
  print(i)

In [ ]:
# Read products data into RDD
products = sc.textFile("/public/retail_db/products")
productsFiltered = products.filter(lambda p: p.split(",")[4] != "")

# Convert the data to (k, v) using product category id as key and the entire product record as value
# For each record in productsFiltered, column 1 is the category id, use it as key to this map.
productsMap = productsFiltered.map(lambda p: (int(p.split(",")[1]), p))

# Group by key which is just category id
productsGBCategory = productsMap.groupByKey()

# Get top N products by price in each category
def getTopNProducts(products, topN):
  return sorted(products, key=lambda k: float(k.split(",")[4]), reverse=True)[:topN]

# Get products with top N prices in each category (N different prices, more than N products)
def getTopNPricedProducts(products, topN):
  import itertools as it
  productPrices = sorted(set(map(lambda p: float(p.split(",")[4]), products)), reverse=True)[:topN]
  productsSorted = sorted(products, key=lambda k: float(k.split(",")[4]), reverse=True)
  return it.takewhile(lambda product: float(product.split(",")[4]) in productPrices, productsSorted)
    

# iterate productsGBCategory, <key=category id, value=records with the same id, type=pyspark.resultiterable.ResultIterable>
topNProductsByCategory = productsGBCategory.flatMap(lambda p: getTopNProducts(list(p[1]), 3))
for i in topNProductsByCategory.take(10):
  print(i)